In [36]:
import os
import numpy as np
from scipy import spatial as sspat
import pandas as pd
import matplotlib.pyplot as plt

In [37]:
EPS = 1e-12


def calculate_relative_L1_error(nodes1 :np.ndarray, values1 :np.ndarray, nodes2 :np.ndarray, values2 :np.ndarray):
    """
    
    1. Map mesh nodes1 -> mesh nodes2 (Nearest Neighbor)
    2. Denominator = MAX(|Vc|, |Vf|)
    3. Metric = Sum( |Diff| / Denom ) / N_nodes1
    """
    if nodes1 is None or nodes2 is None:
        return 0.0

    n = values1.shape[0] * values1.shape[1]
    
    # Map nodes1 to nodes2
    tree = sspat.cKDTree(nodes2)
    dists, idx = tree.query(nodes1, k=1)
    values2_mapped = values2[idx]
    
    # Denominator: Max of absolute values
    denominator = np.maximum(np.abs(values1), np.abs(values2_mapped))
    
    # Numerator: Absolute Difference
    diff = np.abs(values1 - values2_mapped)
    
    # Filter valid comparisons
    mask = denominator > EPS
    
    if np.sum(mask) == 0:
        return 0.0
        
    # Sum of Relative Differences
    valid_terms = diff[mask] / denominator[mask]
    sum_diff = np.sum(valid_terms)
    
    # Normalize by N
    return sum_diff / n

In [38]:
BASE_FOLDER = 'testcases'


def get_test_folders(base_folder=BASE_FOLDER):
    for _, next_folders, _ in os.walk(base_folder):
        return list(map(lambda x: f'{base_folder}/{x}', filter(lambda x: x.startswith('test'), next_folders)))
    return None


def get_test_filename(folder, label):
    for _, _, filenames in os.walk(folder):
        for filename in filenames:
            if label in filename:
                return f'{folder}/{filename}'
    return None


def read_data(filename, coord_cols, vals_cols):
    ds = pd.read_csv(filename)
    return ds[coord_cols], ds[vals_cols]


def get_relative_error(test_folder, coord_cols, vals_cols, matlab_label, ansys_label):
    matlab_filename = get_test_filename(test_folder, matlab_label)
    ansys_filename = get_test_filename(test_folder, ansys_label)
    if matlab_filename is None or ansys_filename is None:
        return None

    matlab_nodes, matlab_values = read_data(matlab_filename, coord_cols, vals_cols)
    ansys_nodes, ansys_values = read_data(ansys_filename, coord_cols, vals_cols)
    return calculate_relative_L1_error(matlab_nodes, matlab_values, ansys_nodes, ansys_values)

In [39]:
DEFAULT_COORD_COLS = ['x', 'y', 'z']
DEFAULT_DISPLACEMENT_VALS_COLS = ['Displacement']
TESTCASES = [
    ('Displacement 10k', 'matlab_10k', 'ansys_10k', DEFAULT_COORD_COLS, DEFAULT_DISPLACEMENT_VALS_COLS),
    ('Displacement 20k', 'matlab_20k', 'ansys_20k', DEFAULT_COORD_COLS, DEFAULT_DISPLACEMENT_VALS_COLS),
    ('Displacement 30k', 'matlab_30k', 'ansys_30k', DEFAULT_COORD_COLS, DEFAULT_DISPLACEMENT_VALS_COLS),
    # strain
    # Von Misses stress
]

In [45]:
MAX_ERR = 0.05

for test_folder in get_test_folders():
    print(f'{test_folder}:')
    for test_name, matlab_label, ansys_label, coord_cols, vals_cols in TESTCASES:
        rel_err = get_relative_error(test_folder, coord_cols, vals_cols, matlab_label, ansys_label)
        prefix = f'    {test_name}'
        if rel_err is not None:
            print(f'{prefix}: {rel_err:.6f}')
            assert rel_err < MAX_ERR
        else:
            print(f'{prefix}: None')
    print()

testcases/test_naca0012:
    Displacement 10k: None
    Displacement 20k: None
    Displacement 30k: None

testcases/test_naca0015:
    Displacement 10k: None
    Displacement 20k: None
    Displacement 30k: None

testcases/test_naca1234:
    Displacement 10k: None
    Displacement 20k: None
    Displacement 30k: None

testcases/test_naca2412:
    Displacement 10k: None
    Displacement 20k: None
    Displacement 30k: None

